In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn

import numpy as np
from sklearn.metrics import r2_score

print("Package is ready.")

Package is ready.


In [2]:
PREDACC = 0
TRIAL = 0
n_steps = 24

l = np.load("./Data/ECtrainset_24.npz")

train_data = l['traindataset']
train_label = l['trainlabel']
test_data = l['testdataset']
test_label = l['testlabel']

mins = l['mins']
maxs = l['maxs']

labelmins = l['labelmins']
labelmaxs = l['labelmaxs']

explanation = "LSTM_24"
DELETE = [3, 4, 5, 11, 12] #Must 3, 4, 5, 11, 12 
EX_PRED = PREDACC #Sorry, it's not automated yet.

train_data = np.delete(train_data, DELETE, axis=1)
test_data = np.delete(test_data, DELETE, axis=1)
train_label = train_label[:,:]
test_label = test_label[:,:]

n_train = train_label.shape[0]
n_test = test_label.shape[0]
n_input = train_data.shape[1]
n_output = test_label.shape[1]
n_classes = n_output


#TRYING decayed learning rate
# learning_rate = 0.001
global_step = tf.Variable(0, trainable=False)
starter_learning_rate = 0.1
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 500, 0.90, staircase=True)

training_epochs = 30000
batch_size = n_train
display_step = 500
n_hidden = 64

x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_output])

weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_output]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_output]))
}

In [3]:
l = np.load("./Data/ECpredset_24.npz")

pred_data = l['preddataset']
pred_label = l['predlabel']
n_pred = pred_label.shape[0]

for i in range(pred_data.shape[1]):
    temp = pred_data[:,i]
    temp = (temp - mins[i]) / (maxs[i] - mins[i])
    pred_data[:,i] = temp

    
for i in range(pred_label.shape[1]):
    temp = pred_label[:,i]
    temp = (temp - labelmins[i]) / (labelmaxs[i] - labelmins[i])
    pred_label[:,i] = temp
    
pred_data = np.delete(pred_data, DELETE, axis=1)

pred_x = pred_data.reshape((-1, n_steps, n_input))

In [4]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, n_steps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

In [5]:
# Define loss and optimizer
cost = tf.multiply(tf.reduce_sum(tf.square(tf.subtract(pred, y))),1/batch_size)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost, global_step=global_step)

# Evaluate model
ymean = tf.reduce_mean(y)
SSE = tf.reduce_sum(tf.square(tf.subtract(y, pred)))
SSR = tf.reduce_sum(tf.square(tf.subtract(pred, ymean)))
r_squared = SSR/(SSE+SSR)
                                
accuracy = r_squared
# Initializing the variables
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [6]:
while EX_PRED < 0.80:
    TRIAL = TRIAL + 1
    EX_PRED = PREDACC
    print("Start again, trial %d" % TRIAL)
    
    sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth =True)))
    sess.run(init)
    
    tf.summary.scalar('RMS', cost)
    tf.summary.scalar('accuracy', accuracy)
    merged = tf.summary.merge_all()
    directory_name = "./tb_logs/" + explanation + str(n_hidden)
    summary_writer = tf.summary.FileWriter(directory_name, graph=sess.graph)
    
    
    # Launch the graph
    num_data = train_data.shape[0]
    ex_pred_acc = EX_PRED

    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(num_data/batch_size/n_steps)

        for i in range(total_batch):
            batch_x = train_data[n_steps*batch_size*i:n_steps*batch_size*(i+1), :]
            batch_y = train_label[batch_size*i:batch_size*(i+1), :]

            # Reshape data
            batch_x = batch_x.reshape((batch_size, n_steps, n_input))
            # Run optimization op (backprop)
            summary, _ = sess.run([merged, optimizer], feed_dict={x: batch_x, y: batch_y})
            avg_cost += sess.run(cost, feed_dict={x: batch_x, y: batch_y})/total_batch
            summary_writer.add_summary(summary, epoch*total_batch+i)

            pred_acc = sess.run(accuracy, feed_dict={x: pred_x, y: pred_label})

            if pred_acc > ex_pred_acc:
                bestpath = "./models/best_trained_" + explanation + ".ckpt"
                save_path = saver.save(sess, bestpath)
                ex_pred_acc = pred_acc

            if epoch % display_step == 0:
                print ("Epoch: %05d/%05d cost: %.6f, best: %.3f" % (epoch, training_epochs, avg_cost,ex_pred_acc))
                train_acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})

                test_x = test_data.reshape((-1, n_steps, n_input))
                test_acc = sess.run(accuracy, feed_dict={x: test_x, y: test_label})
                pred_acc = sess.run(accuracy, feed_dict={x: pred_x, y: pred_label})
                print ("Training Acc: %.3f, Validation Acc: %.3f, Test Acc: %.3f" % (train_acc, test_acc, pred_acc))


    print ("Epoch: %05d/%05d cost: %.6f, best: %.3f" % (epoch+1, training_epochs, avg_cost, ex_pred_acc))
    train_acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})

    test_x = test_data.reshape((-1, n_steps, n_input))
    test_acc = sess.run(accuracy, feed_dict={x: test_x, y: test_label})
    print ("Training Acc: %.3f, Validation Acc: %.3f, Test Acc: %.3f" % (train_acc, test_acc, pred_acc))

    
    #Choose best model
    bestpath = "./models/best_trained_" + explanation + ".ckpt"
    saver.restore(sess, bestpath)
    # print("Model restored, best Acc: %.3f" % ex_pred_acc)

    PREDACC = sess.run(accuracy, feed_dict={x: pred_x, y: pred_label})
    sess.close()

print("Done.")

Start again, trial 1
Epoch: 00000/30000 cost: 672.611084, best: 0.500
Training Acc: 0.500, Validation Acc: 0.500, Test Acc: 0.500
Epoch: 00500/30000 cost: 0.210518, best: 0.500
Training Acc: 0.786, Validation Acc: 0.780, Test Acc: 0.279
Epoch: 01000/30000 cost: 0.160131, best: 0.500
Training Acc: 0.839, Validation Acc: 0.830, Test Acc: 0.342
Epoch: 01500/30000 cost: 0.194548, best: 0.500
Training Acc: 0.807, Validation Acc: 0.798, Test Acc: 0.436
Epoch: 02000/30000 cost: 0.128765, best: 0.500
Training Acc: 0.871, Validation Acc: 0.859, Test Acc: 0.417
Epoch: 02500/30000 cost: 0.118832, best: 0.500
Training Acc: 0.881, Validation Acc: 0.867, Test Acc: 0.445
Epoch: 03000/30000 cost: 0.112382, best: 0.508
Training Acc: 0.890, Validation Acc: 0.876, Test Acc: 0.495
Epoch: 03500/30000 cost: 0.098386, best: 0.533
Training Acc: 0.902, Validation Acc: 0.888, Test Acc: 0.524
Epoch: 04000/30000 cost: 0.095367, best: 0.560
Training Acc: 0.903, Validation Acc: 0.891, Test Acc: 0.521
Epoch: 04500/3

Epoch: 07500/30000 cost: 0.051292, best: 0.668
Training Acc: 0.949, Validation Acc: 0.943, Test Acc: 0.613
Epoch: 08000/30000 cost: 0.044211, best: 0.668
Training Acc: 0.956, Validation Acc: 0.948, Test Acc: 0.623
Epoch: 08500/30000 cost: 0.041943, best: 0.668
Training Acc: 0.958, Validation Acc: 0.949, Test Acc: 0.647
Epoch: 09000/30000 cost: 0.038053, best: 0.668
Training Acc: 0.962, Validation Acc: 0.953, Test Acc: 0.648
Epoch: 09500/30000 cost: 0.036107, best: 0.668
Training Acc: 0.964, Validation Acc: 0.955, Test Acc: 0.647
Epoch: 10000/30000 cost: 0.034876, best: 0.668
Training Acc: 0.965, Validation Acc: 0.956, Test Acc: 0.651
Epoch: 10500/30000 cost: 0.033452, best: 0.668
Training Acc: 0.966, Validation Acc: 0.958, Test Acc: 0.652
Epoch: 11000/30000 cost: 0.031186, best: 0.668
Training Acc: 0.969, Validation Acc: 0.960, Test Acc: 0.647
Epoch: 11500/30000 cost: 0.029697, best: 0.668
Training Acc: 0.970, Validation Acc: 0.962, Test Acc: 0.648
Epoch: 12000/30000 cost: 0.027918, be

KeyboardInterrupt: 